In [ ]:
library(dplyr)
library(Dict)
library(outbreakinfo)
authenticateUser()


In [ ]:

states=c("New York","Arizona", "Minnesota") #THESE STATES, 
states=c("Connecticut","Iowa","Minnesota","Michigan","North Dakota", "New Jersey", "Oklahoma", "Pennsylvania", "Rhode Island", "Wisconsin","New York","Arizona","South Dakota")
states=c("Minnesota","Michigan","North Dakota", "New Jersey", "Oklahoma", "Pennsylvania", "Rhode Island", "Wisconsin","New York","Arizona","South Dakota")



In [ ]:

length(states)
#Pulling lineages for each of the states above. 
for (state in states){
  print(state)
  
  df = getAllLineagesByLocation(location = state, other_threshold=0.00001, nday_threshold=50, ndays = 1300, cumulative = F)
  write.csv(df, paste("./",state,"_raw-01.csv", sep = ''))
}

lineage_dat_dic = getCuratedLineages()

`%like%` <- function (x, pattern) { 
  pattern_fix <-paste0('\\b',pattern,'\\b')
  grepl(pattern_fix, x, ignore.case=TRUE)
}

## Adding WHO names to each of the variants see in pull above. 
for (state in states){
  print(paste("##############",state,"##############"))
  temp = read.csv(paste("./",state,"_raw-01.csv", sep = ''))
  aa<-c()
  dd <- Dict$new(
    info = 'Dict to speed up search time.'
  )


  for (i in trimws(temp$lineage)){
    
    if (is.null(dd$get(i))){
        #a<-which(lineage_dat_dic$searchTerms %like% i)
        #a<-which(lineage_dat_dic$reportQuery$pango %like% i)
        
        #for (n in range(1:length(lineage_dat_dic$reportQuery$pango):1)){
        for (n in seq_len(nrow(lineage_dat_dic))){
          #print(n)
          if (i %in% tolower(lineage_dat_dic$reportQuery$pango[[n]])){
            term=lineage_dat_dic$label[n]
            print(paste(i,term, sep=' - '))
            dd[i] <- term
            aa = append(aa,term)
          }
          else{
            if(i != ""){
              dd[i]=NA
            }
        }
        
        

        }

        aa = append(aa,NA)
      }
      else{
        aa = append(aa,dd$get(i))
      }
      
    }

temp$term <- aa
write.csv(temp, paste("./",state,"_data-pangoterms.csv", sep = ''))

}